# Practical 5: Introduction to exiobase

In this practical, you will learn how to load and work with exiobase

You can download exiobase through this link https://exiobase.eu/index.php/data-download/exiobase3mon?limit=20&limitstart=20
 
We will work with: EXIOBASE 3.4 - IOT - 2011 - pxp

The objectives of the practical are:
- See the data available within exiobase
- Calculate footprints using exiobase
- Make a visualization

## Exercise 1: Load the data

Import the IO exiobase data for the year 2011 in product-by-product format

In [1]:
# Import modules
import pandas as pd
import numpy as np

#### 1.1 Import exiobase

Beware: exiobase is composed by large datasets so it may take some time to load and process

In [2]:
A = pd.read_csv("data/IOT_2019_pxp/IOT_2019_pxp/A.txt", delimiter="\t", index_col=[0, 1], header=[0, 1]) # Z matrix
Y = pd.read_csv("data/IOT_2019_pxp/IOT_2019_pxp/Y.txt", delimiter="\t", index_col=[0,1], header=[0,1])  # y matrix
F = pd.read_csv("data/IOT_2019_pxp/IOT_2019_pxp/impacts/F.txt", delimiter="\t", index_col=[0], header=[0,1])  # satellite matrix
F_hh = pd.read_csv("data/IOT_2019_pxp/IOT_2019_pxp/impacts/F_Y.txt", delimiter="\t", index_col=[0], header=[0,1]) # satellite for FD matrix

#### 1.2 Look at the available labels in exiobase
You may do this by printing the labels of your imported matrices or by opening the following files in your data folder:
- finaldemands.txt
- products.txt
- satellite/unit.txt

Since we don't have a file showing all individual regions. 
Here is a code example of how you can get a list of all the regions within exiobase

In [3]:
# First we collect all labels from A
A_labels = A.index
A_labels

MultiIndex([('AT',                                          'Paddy rice'),
            ('AT',                                               'Wheat'),
            ('AT',                                   'Cereal grains nec'),
            ('AT',                             'Vegetables, fruit, nuts'),
            ('AT',                                           'Oil seeds'),
            ('AT',                              'Sugar cane, sugar beet'),
            ('AT',                                  'Plant-based fibers'),
            ('AT',                                           'Crops nec'),
            ('AT',                                              'Cattle'),
            ('AT',                                                'Pigs'),
            ...
            ('WM',                       'Paper for treatment: landfill'),
            ('WM',               'Plastic waste for treatment: landfill'),
            ('WM', 'Inert/metal/hazardous waste for treatment: landfill'),
         

In [4]:
# .to_frame() to turn the collected labels into a dataframe
A_labels = A_labels.to_frame(index=None)
A_labels

,region,sector
0,AT,Paddy rice
1,AT,Wheat
2,AT,Cereal grains nec
3,AT,"Vegetables, fruit, nuts"
4,AT,Oil seeds
...,...,...
9795,WM,Membership organisation services n.e.c. (91)
9796,WM,"Recreational, cultural and sporting services (92)"
9797,WM,Other services (93)
9798,WM,Private households with employed persons (95)


A_labels is composed by two columns "region" and "sector"
by doing 

> A_labels.region 

or 

> A_labels.sector 

you can access the specific columns 

N.b.
it is the equivalent of doing 

> A_labels.loc[:, "region"]

How do we know how many regions, sectors or categories do we have available?

In [5]:
# Then we extract region column and eliminate any duplicate labels
# We do this because the labels are replicated for each sectoral category in each region
regions_labels = A_labels.region.unique()

# We print the regional labels so that we can see the regions we have to work with
regions_labels.size

49

In [6]:
sectors_labels = A_labels.sector.unique()

# Print the sectoral labels
sectors_labels.size

200

## Exercise 2: Calculate the rest of the IO variables (I, L, x)

#### 2.1 First we calculate the Leontief inverse

In [7]:

I = np.identity(A.shape[0]) # A.shape[0] is the total number of columns in the A matrix
L = np.linalg.inv(I-A)
L

array([[1.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 1.03428586e+00, 1.25597428e-05, ...,
        7.92387971e-07, 3.20440448e-07, 0.00000000e+00],
       [0.00000000e+00, 3.69204131e-05, 1.01458483e+00, ...,
        2.46470027e-06, 5.24856750e-07, 0.00000000e+00],
       ...,
       [0.00000000e+00, 4.62181987e-06, 4.13319935e-06, ...,
        1.01367267e+00, 2.84431457e-04, 0.00000000e+00],
       [0.00000000e+00, 8.40582312e-06, 7.91799006e-06, ...,
        3.67771108e-04, 1.00416733e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 1.00000000e+00]])

#### 2.2 We calculate our product output x

In [8]:
x = L@Y.sum(axis=1)

# A check to see whether the resulting x is right
print(x.shape)
print(x)

(9800,)
[    0.           359.79773658   864.54809943 ... 35533.35519031
  6245.82863651     0.        ]


## Exercise 3: Create a matrix of extension intensities 

When working with real data, you will find cases in which the produt output vector x contains 0's 

If you try to invert the diagonalized product output, it will tell you that you cannot invert a Singular matrix.

There are various reasons why a matrix may be singular but in our case it is due to the fact that not all values are non-zeros

You may then be tempted to perform 1/x, however, this may results in several divisions by 0's and the resulting matrix will be filled with NaN values or inf values.

One way to get around this: Divide 1 by the values that are non-0 as shown in the following example

In [9]:
# we make a copy of our product output vector
x_ = x
# we divide 1 by the values that are non-0
#x_ = 1/x_, then replace inf values
x_[x_!=0]= 1/x_[x_!=0] #calculates 1/x only for the number that are  not zero 
print(x_)
# We diagolize the resulting vector
inv_diag_x = np.diag(x_)
print(F.shape)
# We are essentially dividing the total extension by the product output
# This gives us coefficients of extension by unit of output (e.g., kg/euro)
f = F @ inv_diag_x

f

[0.00000000e+00 2.77933933e-03 1.15667364e-03 ... 2.81425718e-05
 1.60106858e-04 0.00000000e+00]
(126, 9800)


,0,1,2,3,4,5,6,7,8,9,...,9790,9791,9792,9793,9794,9795,9796,9797,9798,9799
impact,,,,,,,,,,,,,,,,,,,,,
Value Added,0.0,6.758705e-01,0.742589,0.690073,8.318040e-01,5.240212e-01,0.931297,0.548421,3.762497e-03,0.471794,...,3.764269e-01,3.512124e-01,0.389520,3.828206e-01,3.538083e-01,0.479711,0.535663,0.550831,0.717995,0.0
Employment,0.0,2.162651e-02,0.016105,0.022381,2.495181e-02,2.528435e-02,0.000603,0.003960,3.848932e-02,0.024352,...,5.706826e-03,7.018623e-03,0.005744,8.103717e-03,7.628601e-03,0.007756,0.006710,0.005708,0.014335,0.0
Employment hour,0.0,4.759831e+04,36782.612201,50517.645427,5.897691e+04,6.203679e+04,1484.470666,8962.473079,9.796496e+04,59816.871185,...,1.532442e+04,1.884651e+04,15059.926003,2.116965e+04,1.957432e+04,21628.179545,18372.663175,15610.512899,40895.270100,0.0
"GHG emissions (GWP100) | Problem oriented approach: baseline (CML, 2001) | GWP100 (IPCC, 2007)",0.0,1.690932e+06,935119.414396,181589.602023,1.301896e+06,1.649214e+06,347941.086195,957782.044611,4.970012e+06,465137.987386,...,2.023481e+07,1.390853e+06,172467.064319,8.445838e+06,1.851824e+06,24267.534015,57159.204690,147562.200164,10687.150229,0.0
"Human toxicity (USEtox) | USEtox2008 | CTUh (Rosenbaum et al., 2008)",0.0,6.446307e-03,0.002427,0.000605,3.198484e-03,1.566391e-03,0.000049,0.000911,2.602296e-03,0.001573,...,8.941858e-02,9.517493e-02,0.141794,1.236203e-01,1.049235e-01,0.008565,0.008359,0.021177,0.000168,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Unused Domestic Extraction - Oil and Gas,0.0,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000000,...,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.0
Unused Domestic Extraction - Non-metalic Minerals,0.0,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000000,...,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.0
Unused Domestic Extraction - Iron Ore,0.0,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000000,...,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.0


N.b. inverting a matrix is a more complex operation than just dividing 1 by the values in your matrix. 

However, in the case of a diagonalized vector with non-zero values along the diagonal 1/diag(x) and inv(diag(x)) output the same results. 

If you have zero's in x vector to be diagonalized then you will not be able to perform the inversion.    

## Exercise 4: Total footprint of the Netherlands


- *What is the total carbon footprint of the Netherlands?*


$\text{F} = \text{f} \mathbf{L}\text{Y} + \text{F}_{hh}$

#### 4.1 We first create a (modified) final demand matrix

4.1.1 Lets identify the range of the Y columns concerning the Netherlands

In [10]:
# we know NL is the 20th country (python counting starting from 0) in the list of countries 
# and that we have 7 final demand categories, therefore

start_NL = 140
end_NL = 147


4.1.2 We calculate the modified Y

You can slice your Y by using pandas iloc method 

> Y.iloc[:,start_NL:end_NL]

In [11]:
Y.iloc[: , 140:147]

region                                                                                            NL  \
category                                                 Final consumption expenditure by households   
region sector                                                                                          
AT     Paddy rice                                                                       0.000000e+00   
       Wheat                                                                            8.109941e-01   
       Cereal grains nec                                                                7.538257e-01   
       Vegetables, fruit, nuts                                                          3.635396e+00   
       Oil seeds                                                                        3.967363e-09   
...                                                                                              ...   
WM     Membership organisation services n.e.c. (91)                                     1.313962e-03   
       Recreational, cultural and sporting services (92)                                0.000000e+00   
       Other services (93)                                                              1.449982e+00   
       Private households with employed persons (95)                                    0.000000e+00   
       Extra-territorial organizations and bodies                                       0.000000e+00   

region                                                                                                                                         \
category                                                 Final consumption expenditure by non-profit organisations serving households (NPISH)   
region sector                                                                                                                                   
AT     Paddy rice                                                                                  0.000000                                     
       Wheat                                                                                       0.000000                                     
       Cereal grains nec                                                                           0.000000                                     
       Vegetables, fruit, nuts                                                                     0.000000                                     
       Oil seeds                                                                                   0.000000                                     
...                                                                                                     ...                                     
WM     Membership organisation services n.e.c. (91)                                                0.252829                                     
       Recreational, cultural and sporting services (92)                                           0.747362                                     
       Other services (93)                                                                         0.000000                                     
       Private households with employed persons (95)                                               0.000000                                     
       Extra-territorial organizations and bodies                                                  0.000000                                     

region                                                                                                \
category                                                 Final consumption expenditure by government   
region sector                                                                                          
AT     Paddy rice                                                                           0.000000   
       Wheat                                                                                0.00

Or by using the labels through pandas loc method

> Y.loc[:, "NL"]

In [29]:
Y_mod = Y.loc[:, "NL"]
Y_mod

category                                                  Final consumption expenditure by households  \
region sector                                                                                           
AT     Paddy rice                                                                        0.000000e+00   
       Wheat                                                                             8.109941e-01   
       Cereal grains nec                                                                 7.538257e-01   
       Vegetables, fruit, nuts                                                           3.635396e+00   
       Oil seeds                                                                         3.967363e-09   
...                                                                                               ...   
WM     Membership organisation services n.e.c. (91)                                      1.313962e-03   
       Recreational, cultural and sporting services (92)                                 0.000000e+00   
       Other services (93)                                                               1.449982e+00   
       Private households with employed persons (95)                                     0.000000e+00   
       Extra-territorial organizations and bodies                                        0.000000e+00   

category                                                  Final consumption expenditure by non-profit organisations serving households (NPISH)  \
region sector                                                                                                                                    
AT     Paddy rice                                                                                  0.000000                                      
       Wheat                                                                                       0.000000                                      
       Cereal grains nec                                                                           0.000000                                      
       Vegetables, fruit, nuts                                                                     0.000000                                      
       Oil seeds                                                                                   0.000000                                      
...                                                                                                     ...                                      
WM     Membership organisation services n.e.c. (91)                                                0.252829                                      
       Recreational, cultural and sporting services (92)                                           0.747362                                      
       Other services (93)                                                                         0.000000                                      
       Private households with employed persons (95)                                               0.000000                                      
       Extra-territorial organizations and bodies                                                  0.000000                                      

category                                                  Final consumption expenditure by government  \
region sector                                                                                           
AT     Paddy rice                                                                            0.000000   
       Wheat                                                                                 0.000000   
       Cereal grains nec                                                                     0.000000   
       Vegetables, fruit, nuts                                                               0.000000   
       Oil seeds                                                                             0.000000   


#### 5.1 First we isolate the extension in which we are interested

For this exercise we only focus on one component of the carbon fooprint

*"CO2 - combustion - air"* in kg

In [13]:
# the intensity vector in which we are interested
indicator = "GHG emissions (GWP100) | Problem oriented approach: baseline (CML, 2001) | GWP100 (IPCC, 2007)"
f_ = f.loc[indicator]
f_

0       0.000000e+00
1       1.690932e+06
2       9.351194e+05
3       1.815896e+05
4       1.301896e+06
            ...     
9795    2.426753e+04
9796    5.715920e+04
9797    1.475622e+05
9798    1.068715e+04
9799    0.000000e+00
Name: GHG emissions (GWP100) | Problem oriented approach: baseline (CML, 2001) | GWP100 (IPCC, 2007), Length: 9800, dtype: float64

In [16]:
# the final demand CO2 emissions

F_hh_ = F_hh.loc[indicator]

#### 5.2 We calculate the total footprint of the region

In [31]:
# Calculate the total global footprint
# I cannot add + F_hh_
F_total = np.diag(f_) @ L @ Y_mod 
F_total.size

68600

## Exercise 5: Which regions emit the most CO2 as a result of final consumption in the Netherlands?

#### 5.3 Let's analyse in which regions CO2 is emitted the most as a result of NL consumption

In [ ]:
# In this case we diagonalize the emission intensity vector 
F_breakdown = None
F_breakdown

In [ ]:
# we apply the sectoral labels
F_breakdown = None

# we print the results
F_breakdown

We then sum the results by using the groupby and sum methods in combination in the following manner

In [ ]:
# we sum the results by regions
F_regional_breakdown = None

# We sort the results from largest to smallest
F_rb_sorted = None

F_rb_sorted

## Exercise 6: Let's plot the results for the top 15 emitters 

Using pandas you can make simple visualizations directly from dataframes and series

see more here: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.plot.html

#### 6.1 Totals of the top 15 emitters

In [ ]:
# plot your results with plot.bar()

#### 6.2 Let's normalize results by the total footprint of NL consumption

In [ ]:
# Normalize your results
F_rb_sorted_norm = None

# Plot top 15 regions
